# 🔍 Enricher for TTD Newsletter

This notebook is used to find, test, and evaluate enriching methods over articles.

In [1]:
import sys
import os
sys.path.append('../')
from ttd.storage.ttd_storage import TTDStorage
from dotenv import load_dotenv
from ttd.config import load_config
from pprint import pprint

load_dotenv()
OPEN_ROUTER_API_KEY = os.getenv("OPEN_ROUTER_API_KEY")
config = load_config()
DB_PATH = '../' + config["db_path"]

In [2]:
storage = TTDStorage(DB_PATH)
articles = storage.get_all("articles")

In [3]:
pprint(list(articles[0].keys()))

['title',
 'author',
 'published_date',
 'url_domain',
 'url',
 'summary',
 'tags',
 'created_at',
 'html_content_path',
 'text_content_path']


In [ ]:
model = {
    "name": "deepseek-r1-distill-qwen-32b:free",
    "version": "2025-01-29",
    "input_format": "text",
    "output_format": "text",
    "task_type": "summarization",
    "config": {
        "openai": {
            "base_url": "https://openrouter.ai/api/v1",
            "api_key_env_var": "OPEN_ROUTER_API_KEY",
            "temperature": 0,
            "max_tokens": 256,
            "n": 1
        }
    }
}
storage.save_model(model)

In [4]:
model_dict = storage.get_model_by_name("deepseek-r1-distill-qwen-32b:free")
model_dict["config"]["openai"]["model"] = "deepseek/deepseek-r1-distill-qwen-32b:free"
model_dict["config"]["openai"]["max_tokens"] = None
storage.update_model(model_dict)

In [5]:
model_dict = storage.get_model_by_name("deepseek-r1-distill-qwen-32b:free")
pprint(model_dict)

{'config': {'openai': {'api_key_env_var': 'OPEN_ROUTER_API_KEY',
                       'base_url': 'https://openrouter.ai/api/v1',
                       'max_tokens': None,
                       'model': 'deepseek/deepseek-r1-distill-qwen-32b:free',
                       'n': 1,
                       'temperature': 0}},
 'created_at': '2025-04-01T20:03:35.982601',
 'input_format': 'text',
 'last_updated': '2025-04-01T21:04:56.757096',
 'name': 'deepseek-r1-distill-qwen-32b:free',
 'output_format': 'text',
 'task_type': 'summarization',
 'version': '2025-01-29'}


In [6]:
model = storage.load_model_by_name("deepseek-r1-distill-qwen-32b:free")
model

In [7]:
model.model_name,model.config

('deepseek/deepseek-r1-distill-qwen-32b:free',
 {'temperature': 0, 'max_tokens': None, 'n': 1})

In [9]:
model.predict("Will Train, Tune, Deploy - AI Newsletter Service be a succes ?")

'The "Will Train, Tune, Deploy - AI Newsletter Service" has the potential to be successful, particularly if it effectively addresses the needs of its target audience and overcomes several key challenges. Here\'s a structured analysis of its potential success factors and challenges:\n\n### Potential Success Factors:\n\n1. **Market Demand**: AI is a growing field with increasing interest from businesses and individuals. The service targets those who need AI solutions but lack the expertise, suggesting a viable market.\n\n2. **Comprehensive Offerings**: By providing training, tuning, and deployment services, the platform covers all stages of AI model development, which can be appealing to clients looking for a one-stop solution.\n\n3. **Target Audience**: Small to medium businesses, startups, and individuals with some technical background are likely to benefit, as they may lack the resources for in-house AI development.\n\n4. **Content Value**: If the newsletter offers exclusive insights 